# How to build the h-net
## Prepare environment and  define parameters

In [ ]:
is_colab = True
import sys
if is_colab:
    !pip install -q torch==1.9.0
    !pip install -q torchvision==0.10.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz

import torch
import torchvision

import qiskit
from qiskit import  QuantumCircuit, ClassicalRegister
import numpy as np
import functools


from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.n_lyr_circ import N_LYR_Circ
from qfnn.qf_circ.u_lyr_circ import U_LYR_Circ
from qfnn.qf_circ.p_lyr_circ import P_LYR_Circ
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix
from qfnn.qf_net.utils import binarize
from qfnn.qf_fb.c_qf_mixer import Net
from qfnn.qf_fb.c_input import ToQuantumData
print = functools.partial(print, flush=True)

## Inference
### Input data preparation

In this example, we will use MNIST dataset for demonstrating how the neural network
runs on a quantum circuit.

So, the first step is to load data from the MNIST. There are serveral points to be
 noted.

* The model trained for demonstration is based on the subset of MNSIT with digits 3 and 6.
* The input is down sampled to 4 by 4 to be executed on 4 qubits (which has 2^4=16 states).

In [ ]:
interest_num = [3,6]
img_size = 4
batch_size = 32
inference_batch_size = 1

if is_colab:
    data_path = '/content/data' #mnist  path
else:
    data_path = 'Your path for data' #mnist  path

train_loader, test_loader = load_data(interest_num,data_path,False,img_size,batch_size,inference_batch_size,False)
for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix = to_quantum_matrix(data)
    break

### load model

We have trained the neural network model. Here, we need to extract the data from the
model to the local variables, which will be used for building the quantum circuit.

In [ ]:
if is_colab:
    resume_path = '/content/model/u2_p2/model_best.tar' #model path
else:
    resume_path = 'Your path for model' #model path

checkpoint = torch.load(resume_path, map_location='cpu')
print(checkpoint['state_dict']['fc0.weight'])
print(checkpoint['state_dict']['fc1.weight'])
# print(checkpoint['state_dict']['fc2.batch.x_l_0_5'])
# print(checkpoint['state_dict']['fc2.batch.x_g_0_5'])

weight_1 = checkpoint['state_dict']['fc0.weight']
weight_2 = checkpoint['state_dict']['fc1.weight']
# norm_flag = checkpoint['state_dict']['fc2.batch.x_l_0_5']
# norm_para = checkpoint['state_dict']['fc2.batch.x_running_rot']
# print(norm_para)

## Build the QF-pNet network

In the following cell, we build the first layer of the QF-hNet using the qfnn library.


In [ ]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of u-layer
######################################################
#create circuit
circuit = QuantumCircuit()

#init circuit, which is corresponding to a neuron with 4 qubits and 2 outputs
u_layer = U_LYR_Circ(4,2)

#create qubits to be invovled
inps = u_layer.add_input_qubits(circuit)
aux =u_layer.add_aux(circuit)
u_layer_out_qubits = u_layer.add_out_qubits(circuit)

#add u-layer to your circuit
u_layer.forward(circuit,binarize(weight_1) ,inps,u_layer_out_qubits,quantum_matrix,aux)

#show your circuit
circuit.barrier()
circuit.draw('text',fold=300)

Now, we have a one-layer neural network with U-LYR. In the next step, we will on top
of this layer to add an output layer, which is based on P-LYR, to the built
quantum circuit of QF-hN.

In [ ]:
#add p-layer to your circuit
p_layer = P_LYR_Circ(2,2)
p_aux = p_layer.add_aux(circuit,"p_aux")
p_layer_output = p_layer.add_out_qubits(circuit)
p_layer.forward(circuit,binarize(weight_2),u_layer_out_qubits,p_layer_output,p_aux)

# #add n-layer to your circuit
# norm = N_LYR_Circ(2)
# norm_qubit = norm.add_norm_qubits(circuit)
# norm_output_qubit = norm.add_out_qubits(circuit)
# norm.forward(circuit,p_layer_output,norm_qubit,norm_output_qubit,norm_flag,norm_para)

#add measurement to your circuit
add_measure(circuit,p_layer_output,'reg')

print("Output layer created!")

circuit.draw('text',fold =300)

### Quantum simulation

In [ ]:
################ Weiwen on 12-30-2020 ################
# Quantum simulation
######################################################

qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)


print("Simulation Result :",opt_class_prob)
print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
print("Target class: {}".format(target[0]))
if opt_class_prob.index(max(opt_class_prob))==target[0]:
    print("Correct prediction")
else:
    print("Incorrect prediction")
print("="*30)

### classical inference

In [ ]:

neural_in_layers = 'u:2,p:2'
layers = []
for item1 in neural_in_layers.split(","):
    x= item1.split(":")
    layer =[]
    layer.append(x[0].strip())
    layer.append(int(x[1].strip()))
    layers.append(layer)
given_ang =[]
given_ang.append([])
given_ang.append([])
# given_ang.append(norm_para)
model = Net(img_size,layers,False,False,given_ang,False)
model.load_state_dict(checkpoint["state_dict"])
# print(quantum_matrix)
to_quantum_data = ToQuantumData(img_size)
output_data = to_quantum_data(data)
output = model.forward(output_data,False)
print("classical inference:",output)